In [2]:
import requests
import os
import time
import numpy as np
import logging
import sys

def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(1) # for a period defined by a Gaussian with mu=1 and sigma=0.25.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments

    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    #header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']

    self.project_logger = logging.getLogger()
    self.project_logger.setLevel(logging.INFO)

    log_mode = 'w' if (overwrite_log or not os.path.isfile(logfile)) else 'a'
    self.handler = logging.FileHandler(logfile, mode=log_mode)
    self.handler.setLevel(logging.INFO)
    self.formatter = logging.Formatter('%(message)s')
    self.handler.setFormatter(self.formatter)
    self.project_logger.addHandler(self.handler)

    if overwrite_log == True or not os.path.isfile(logfile):
      # Write header to log file
      self.project_logger.info(
        message=self.log_message('id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error'))

    with open(logfile,'r') as f: # open file
      l = f.readlines()
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1].split(';')[0]) + 1

  def log_message(self, project, t, delta_t, url, redirect_url, response_size, response_code, success, error):
    return '{};{};{};{};{};{};{};{};{};{};{}'.format(self.id, project, self.connector_type, t, delta_t, url, redirect_url, response_size, response_code, success, error)
    
  def _inc_id(self):
    self.id += 1

  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling
          response = self.session.get(url,timeout = self.timeout) # make get call
          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = time.time() - t # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          self._inc_id() # increment call id
          self.project_logger.info(
              msg=self.log_message(project_name,t,dt,url,redirect_url,size,response_code,success,err))
          return response, self.id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt =  time.time() - t # define delta t
          ## log...
          self._inc_id() # increment call_id
          self.project_logger.info( \
              msg=self.log_message(project_name,t,dt,url,redirect_url,size,response_code,success,err))
    else:
      ratelimit()
      t = time.time()
      self.browser.get(url) # use selenium get method
      ## log
      self._inc_id() # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = time.time() - t # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      self.project_logger.info( \
          msg=self.log_message(project_name,t,dt,url,redirect_url,size,response_code,success,err))
      # Using selenium it will not return a response object, instead you should call the browser object of the connector.
      return self.id

In [2]:
### Read in the file products.txt, assumed one product per line
with open('products.txt', 'r') as f:
    products = f.readlines()

### Remove trailing new lines
products = [p.rstrip() for p in products]
print(str(products))

['panodil']


In [ ]:
### Reads in a list of product numbers from

In [3]:
import requests

conn = Connector('medicin_prices.log')

def get_product_info(product_name: str):
    # "http://api.medicinpriser.dk/v1/produkter/virksomtstof/Mianserin?format=json"
    info_url = 'http://api.medicinpriser.dk/v1/produkter/{}?format=json'
    response, _ = conn.get(info_url.format(product_name), 'med-scrape')
    return response


In [4]:
def get_product_details(product_number):
    details_url = 'http://api.medicinpriser.dk/v1/produkter/detaljer/{}?format=json'
    response, _ = conn.get(details_url.format(product_number), 'med-scrape')
    return response

In [5]:
# start by defining a Project name
project = 'medicin_prices'
import os # generel package for interacting with the system
# among other things automate folder creation
def maybe_create_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

maybe_create_dir(project)

subfolders = ['raw_data','parsed_data']

raw_data = os.path.join(project, subfolders[0])
parsed_data = os.path.join(project, subfolders[1])

for directory in subfolders: 
    maybe_create_dir(os.path.join(project, directory))

In [6]:
import os

def has_product_info(vnr):
    return os.path.isfile(os.path.join(project, vnr + '.json'))

In [7]:
products[0]

'panodil'

In [8]:
p_info = [get_product_info(pn) for pn in products if not has_product_info(pn)]

In [9]:
p_info

[<Response [200]>]

In [10]:
print(str(p_info))
print([p.text for p in p_info])

[<Response [200]>]
['[{"Navn":"Panodil Zapp","Varenummer":"005775","Firma":"GlaxoSmithKline Consumer","Styrke":"500 mg","Detaljer":"/v1/produkter/detaljer/005775","Pakning":"10 stk. (blister) filmovertrukne tabl."},{"Navn":"Panodil","Varenummer":"005813","Firma":"GlaxoSmithKline Consumer","Styrke":"500 mg","Detaljer":"/v1/produkter/detaljer/005813","Pakning":"10 stk. (blister) filmovertrukne tabl."},{"Navn":"Panodil Brus","Varenummer":"005814","Firma":"GlaxoSmithKline Consumer","Styrke":"500 mg","Detaljer":"/v1/produkter/detaljer/005814","Pakning":"10 stk. brusetabletter"},{"Navn":"Panodil","Varenummer":"008453","Firma":"GlaxoSmithKline Consumer","Styrke":"665 mg","Detaljer":"/v1/produkter/detaljer/008453","Pakning":"100 stk. (dåse) tabl. m modif udløsn"},{"Navn":"Panodil Zapp","Varenummer":"038252","Firma":"GlaxoSmithKline Consumer","Styrke":"500 mg","Detaljer":"/v1/produkter/detaljer/038252","Pakning":"100 stk. filmovertrukne tabl."},{"Navn":"Panodil","Varenummer":"048744","Firma":"G

In [11]:
import json

product_json = [json.loads(p.text) for p in p_info]

In [12]:
for p_list in product_json:
    for product in p_list:
        vnr = product['Varenummer']
        file_path = os.path.join(raw_data, vnr + '.json')
        with open(file_path, 'w') as f:
            json.dump(product, f)

In [13]:
# Create a flattened list of product numbers to scrape
product_numbers = [product["Varenummer"] for product_list in product_json for product in product_list]

In [14]:
product_details = [get_product_details(product_number) for product_number in product_numbers]

In [15]:
product_details_json = [json.loads(p.text) for p in product_details]

In [16]:
product_details_json

[{'Navn': 'Panodil Zapp',
  'Varenummer': '005775',
  'Styrke': '500 mg',
  'Pakning': '10 stk. (blister) filmovertrukne tabl.',
  'VirksomtStof': 'Paracetamol',
  'Firma': 'GlaxoSmithKline Consumer',
  'AtcKode': 'N02BE01',
  'Dosisdispensering': False,
  'Udleveringsgruppe': 'HX18',
  'PrisPrPakning': None,
  'PrisPrEnhed': None,
  'AIP': None,
  'TilskudBeregnesAf': None,
  'Udgaaet': False,
  'UdgaaetDato': None,
  'Substitutioner': [],
  'BilligereKombinationer': [],
  'Dosering': '2 tabletter 3 gange daglig, 2 tabletter 4 gange daglig',
  'Indikation': 'mod smerter, febernedsættende',
  'TrafikAdvarsel': False,
  'DDD': None,
  'Opbevaringsbetingelser': 'Ingen',
  'NbsSpeciale': '',
  'Haandkoeb': True,
  'TilskudKode': '',
  'TilskudTekst': ''},
 {'Navn': 'Panodil',
  'Varenummer': '005813',
  'Styrke': '500 mg',
  'Pakning': '10 stk. (blister) filmovertrukne tabl.',
  'VirksomtStof': 'Paracetamol',
  'Firma': 'GlaxoSmithKline Consumer',
  'AtcKode': 'N02BE01',
  'Dosisdispenser

In [17]:
for product in product_details_json:
    vnr = product['Varenummer']
    file_path = os.path.join(raw_data, 'details_' + vnr + '.json')
    with open(file_path, 'w') as f:
        json.dump(product, f)

In [18]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

import time

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options, executable_path='../../geckodriver')

def get_medication_price_grid_html(vnr: str, sleep_for=20):
    # Create base_url based on vnr
    base_url = 'https://medicinpriser.dk/Default.aspx?id=15&vnr=' + vnr
    # Fetch the base_url using the firefox webdriver
    driver.get(base_url)
    # Find the price development link
    price_development = driver.find_element_by_id("ctl00_TopBar_Pridudvikling")
    # Click the price development link
    price_development.click()
    # Find the date_from input box
    date_from_input = driver.find_element_by_id('ctl00_ctl07_ctl00_DateFrom')
    # Click the date_from input field
    date_from_input.click()
    # Clear the date_from input field
    date_from_input.clear()
    # Send the '01-01-1998' key stroke to the date_from input field
    date_from_input.send_keys('01-01-1998')
    # Send key <enter> stroke to the date_from input field
    date_from_input.send_keys(Keys.ENTER)
    # Submit the content of the date_from input field
    date_from_input.submit()
    # Sleep for sleep_for seconds (default=20)
    time.sleep(sleep_for)
    # Copy the page_source into a string var
    return_str = driver.page_source
    # Return the string var to caller
    return return_str

In [19]:
print(product_json)

[[{'Navn': 'Panodil Zapp', 'Varenummer': '005775', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '500 mg', 'Detaljer': '/v1/produkter/detaljer/005775', 'Pakning': '10 stk. (blister) filmovertrukne tabl.'}, {'Navn': 'Panodil', 'Varenummer': '005813', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '500 mg', 'Detaljer': '/v1/produkter/detaljer/005813', 'Pakning': '10 stk. (blister) filmovertrukne tabl.'}, {'Navn': 'Panodil Brus', 'Varenummer': '005814', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '500 mg', 'Detaljer': '/v1/produkter/detaljer/005814', 'Pakning': '10 stk. brusetabletter'}, {'Navn': 'Panodil', 'Varenummer': '008453', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '665 mg', 'Detaljer': '/v1/produkter/detaljer/008453', 'Pakning': '100 stk. (dåse) tabl. m modif udløsn'}, {'Navn': 'Panodil Zapp', 'Varenummer': '038252', 'Firma': 'GlaxoSmithKline Consumer', 'Styrke': '500 mg', 'Detaljer': '/v1/produkter/detaljer/038252', 'Pakning': '100 stk. filmovertrukne tabl.'}, {'Navn': '

In [20]:
{vn['Varenummer']: vn['Substitutioner'] for vn in product_details_json}

{'005775': [],
 '005813': [{'Navn': 'Pinex',
   'Varenummer': '091323',
   'Firma': 'Teva (Søborg)',
   'Styrke': '500 mg',
   'Detaljer': '/v1/produkter/detaljer/091323',
   'Pakning': '10 stk. (blister) filmovertrukne tabl.'},
  {'Navn': 'Paracetamol "Orifarm"',
   'Varenummer': '391774',
   'Firma': 'Orifarm Generics',
   'Styrke': '500 mg',
   'Detaljer': '/v1/produkter/detaljer/391774',
   'Pakning': '10 stk. (blister) filmovertrukne tabl.'},
  {'Navn': 'Pamol',
   'Varenummer': '523801',
   'Firma': 'Takeda Pharma A/S',
   'Styrke': '500 mg',
   'Detaljer': '/v1/produkter/detaljer/523801',
   'Pakning': '10 stk. (blister) filmovertrukne tabl.'}],
 '005814': [],
 '008453': [],
 '038252': [],
 '048744': [{'Navn': 'Pinex',
   'Varenummer': '056232',
   'Firma': 'Teva (Søborg)',
   'Styrke': '500 mg',
   'Detaljer': '/v1/produkter/detaljer/056232',
   'Pakning': '100 stk. filmovertrukne tabl.'},
  {'Navn': 'Pinemol',
   'Varenummer': '116637',
   'Firma': '2care4',
   'Styrke': '500 

In [21]:
product_number_substitutions = {vn['Varenummer']: [svn['Varenummer'] for svn in vn['Substitutioner']] for vn in product_details_json}

In [22]:
product_number_substitutions

{'005775': [],
 '005813': ['091323', '391774', '523801'],
 '005814': [],
 '008453': [],
 '038252': [],
 '048744': ['056232', '116637', '576810', '598708'],
 '082830': [],
 '083644': ['056250', '064318', '102474', '170773', '510732'],
 '099233': [],
 '433126': ['568972'],
 '550616': [],
 '556578': [],
 '550624': ['530667'],
 '550673': ['078578'],
 '550731': ['487959'],
 '550830': ['475327'],
 '557907': [],
 '409193': ['056205', '446771', '523152'],
 '111955': ['553430']}

In [23]:
print('Product numbers: ' + ', '.join(product_numbers))

Product numbers: 005775, 005813, 005814, 008453, 038252, 048744, 082830, 083644, 099233, 433126, 550616, 556578, 550624, 550673, 550731, 550830, 557907, 409193, 111955


In [24]:
from tqdm import tqdm_notebook as tqdm # Progressbar module
product_html = []

for i in tqdm(range(0, len(product_numbers))):
    product_html.append((product_numbers[i], get_medication_price_grid_html(product_numbers[i])))

In [25]:
import re
from bs4 import BeautifulSoup as bs

def get_price_grid(html_table):
    soup = bs(html_table)
    price_grid_table = soup.findAll('table', attrs={'id': re.compile(r'.*PriceGrid.*')})
    print(len(price_grid_table))
    if (len(price_grid_table) == 1):
        return price_grid_table[0]
    else:
        return None

In [ ]:
print(', '.join([str(len(l)) for l in product_html]))


print(len(product_html))
# print(len(price_grid_table))
# price_grid_table[0]

In [26]:
product_dict = {pno: get_price_grid(html) for pno, html in product_html}

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1


In [50]:
product_dict

{'005775': <table border="1" cellspacing="0" class="grid" id="ctl00_ctl07_ctl00_PriceGrid" rules="all" style="border-collapse:collapse;">
 <tbody><tr valign="middle">
 <th scope="col">Lægemiddel/<br/>prisperiode</th><th scope="col">Tilskudspris (kr.)</th><th scope="col">Panodil Zapp 005775 (kr.)</th>
 </tr><tr>
 <td>29.12.1997</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>12.01.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>26.01.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>09.02.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>23.02.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>09.03.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>23.03.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>06.04.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>20.04.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>04.05.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>18.05.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>01.06.1998</td><td>-</td><td>-</td>
 <

In [43]:
print(len(product_dict))

19


In [28]:
def make_html(table_html):
    return "<{h}><{b}>{s}</{b}></{h}>".format(h='html', b='body', s=str(table_html).replace(',', '.').replace('-', ''))

In [ ]:
import pandas as pd

price_grid_df = pd.read_html(make_html(tables[0]))

In [46]:
product_dict

{'005775': <table border="1" cellspacing="0" class="grid" id="ctl00_ctl07_ctl00_PriceGrid" rules="all" style="border-collapse:collapse;">
 <tbody><tr valign="middle">
 <th scope="col">Lægemiddel/<br/>prisperiode</th><th scope="col">Tilskudspris (kr.)</th><th scope="col">Panodil Zapp 005775 (kr.)</th>
 </tr><tr>
 <td>29.12.1997</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>12.01.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>26.01.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>09.02.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>23.02.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>09.03.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>23.03.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>06.04.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>20.04.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>04.05.1998</td><td>-</td><td>-</td>
 </tr><tr>
 <td>18.05.1998</td><td>-</td><td>-</td>
 </tr><tr class="odd">
 <td>01.06.1998</td><td>-</td><td>-</td>
 <

In [51]:
import pandas as pd

price_grid_dict = {vnr: pd.read_html(make_html(table))[0] for vnr, table in product_dict.items() if table is not None}

In [52]:
price_grid_dict

{'005775':     Lægemiddel/prisperiode  Tilskudspris (kr.)  Panodil Zapp 005775 (kr.)
 0               29.12.1997                 NaN                        NaN
 1               12.01.1998                 NaN                        NaN
 2               26.01.1998                 NaN                        NaN
 3               09.02.1998                 NaN                        NaN
 4               23.02.1998                 NaN                        NaN
 5               09.03.1998                 NaN                        NaN
 6               23.03.1998                 NaN                        NaN
 7               06.04.1998                 NaN                        NaN
 8               20.04.1998                 NaN                        NaN
 9               04.05.1998                 NaN                        NaN
 10              18.05.1998                 NaN                        NaN
 11              01.06.1998                 NaN                        NaN
 12            

In [88]:
import re

pno_re = re.compile('[0-9]{6}')

for pno, df in price_grid_dict.items():
    pnos = [vnr.group() for vnr in map(pno_re.search, df.columns[2:])]
    df.columns = ['datetime', 'tilskudspris'] + pnos

In [89]:
def rename_columns(df):
    pnos = [vnr.group() for vnr in map(pno_re.search, df.columns[2:])]
    df.columns = ['datetime', 'tilskudspris'] + pnos

In [90]:
def get_series(product_number):
    html = get_medication_price_grid_html(product_number)
    table = get_price_grid(html)
    df = pd.read_html(make_html(table))[0]
    rename_columns(df)
#     print(df[0].head())
    return df[product_number]

In [78]:
def find_missing(p_no, p_nos, df):
    missing = []
    for pno in p_nos:
        if pno not in df.columns:
            missing.append(pno)
    print(str(missing))
    return missing

In [72]:
price_grid_dict['005775'].head()

,datetime,tilskudspris,005775
0,29.12.1997,NaN,NaN
1,12.01.1998,NaN,NaN
2,26.01.1998,NaN,NaN
3,09.02.1998,NaN,NaN
4,23.02.1998,NaN,NaN


In [92]:
for pno, df in price_grid_dict.items():
    print(pno)
#     print(df.head())
    if len(product_number_substitutions[pno]) > 0:
#         print(type(df))
#         print(df.columns)
        missing_pnos = find_missing(pno, product_number_substitutions[pno], df)
        if missing_pnos:
            for p in missing_pnos:
                missing_series = get_series(p)
                price_grid_dict[pno][p] = missing_series

005775
005813
[]
005814
008453
038252
048744
[]
082830
083644
['510732']
1
099233
433126
[]
550616
556578
550624
[]
550673
[]
550731
[]
550830
[]
557907
111955
[]


In [93]:
price_grid_dict['083644'].head()

,datetime,tilskudspris,083644,056250,064318,102474,170773,510732
0,29.12.1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.01.1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26.01.1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,09.02.1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23.02.1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# NaN threshold should always be two, as we should have date and atleast one other entry
syntocinon_df = price_grid_df[0].dropna(axis=0, thresh=2)

In [ ]:
syntocinon_df.head()

In [ ]:
syntocinon_df.tail()

In [ ]:
%matplotlib inline
syntocinon_df.plot()

In [ ]:
syntocinon_df.columns[2:]

In [ ]:
import re

vnr_re = re.compile('[0-9]{6}')
print(syntocinon_df.columns)
result = vnr_re.search(syntocinon_df.columns[3])
result.group()
legend_title = [vnr.group() for vnr in map(vnr_re.search, syntocinon_df.columns[2:])]
syntocinon_df.columns = ['datetime', 'tilskudspris'] + legend_title

In [ ]:
syntocinon_df.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(16,10))

# sns.lineplot(x=syntocinon_df['datetime'], y=[syntocinon_df.columns[2:]], data=syntocinon_df)
sns.lineplot(data=syntocinon_df.drop(['datetime'], axis=1))
# len(syntocinon_df['datetime'])
xtick_labels = syntocinon_df[0:119:int(119/8)]['datetime']
plt.xticks(rotation=45, labels=xtick_labels)
# sns.lineplot(x='datetime', y=syntocinon_df.drop(['datetime'], axis=1), data=syntocinon_df)

